# MiMo-VL Tutorial: Multimodal Analysis with FiftyOne

This tutorial demonstrates how to use the MiMo-VL vision-language models with FiftyOne for various visual understanding tasks.

## 1. Load a Sample Dataset

First, let's load a small UI dataset from the FiftyOne Dataset Zoo.

In [1]:
import fiftyone as fo
from fiftyone.utils.huggingface import load_from_hub

# Load 5 random samples from the WaveUI dataset
dataset = load_from_hub(
    "Voxel51/WaveUI-25k",
    max_samples=10,
    shuffle=True
)

Loading dataset
Importing samples...
 100% |███████████████████| 10/10 [3.1ms elapsed, 0s remaining, 3.3K samples/s]      


Launch the FiftyOne App to visualize the dataset (optional)

In [ ]:
fo.launch_app(dataset)

## 2. Set Up MiMo-VL Integration

Register the MiMo-VL remote zoo model source and load the model.

In [3]:
import fiftyone.zoo as foz

# Register the model source
foz.register_zoo_model_source("https://github.com/harpreetsahota204/MiMo_VL", overwrite=True)

  160.8Kb [49.8ms elapsed, ? remaining, 3.2Mb/s] 
Overwriting existing model source '/home/harpreet/fiftyone/__models__/MiMo-VL'


# Load the MiMo-VL-7B-SFT model

You can also use `XiaomiMiMo/MiMo-VL-7B-RL` or `XiaomiMiMo/MiMo-VL-7B-SFT-GGUF`

In [4]:
model = foz.load_zoo_model(
    "XiaomiMiMo/MiMo-VL-7B-SFT",
    # install_requirements=True, #you can pass this to make sure you have all reqs installed
    )

Fetching 17 files:   0%|          | 0/17 [00:00<?, ?it/s]

.gitattributes:   0%|          | 0.00/1.52k [00:00<?, ?B/s]

README.md:   0%|          | 0.00/7.39k [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

You have video processor config saved in `preprocessor.json` file which is deprecated. Video processor configs should be saved in their own `video_preprocessor.json` file. You can rename the file or load and save the processor back which renames it automatically. Loading from `preprocessor.json` will be removed in v5.0.


## 3. Visual Question Answering

Ask the model to describe UI screenshots.

In [5]:
model.operation = "vqa"
model.prompt = "Describe this screenshot and what the user might be doing in it."
dataset.apply_model(model, label_field="vqa_results")

 100% |███████████████████| 10/10 [2.2m elapsed, 0s remaining, 0.1 samples/s]    


Note that for any of the following operations you can use a Field which currently exists on your dataset, all you need to do is pass the name of that field in `prompt_field` when you call `apply_model`. For example:

```python
dataset.apply_model(model, prompt_field="<field-name>", label_field="<label-field>")
```

## 4. Object Detection

Detect interactive UI elements with bounding boxes.

In [ ]:
model.operation = "detect"
model.prompt = "Locate the elements of this UI that a user can interact with."
dataset.apply_model(model, label_field="ui_detections")

  80% |███████████████|---|  8/10 [4.7m elapsed, 1.2m remaining, 0.0 samples/s]  

## 5. Optical Character Recognition (OCR)

Extract and locate text in the UI.

In [ ]:
model.operation = "ocr"
model.prompt = "OCR all the text in the user interface."
dataset.apply_model(model, label_field="ocr_results")


## 6. Keypoint Detection

Identify important points in the UI.

In [ ]:
model.operation = "point"
model.prompt = "Point to all the clickable and interactable elements in user interface."
dataset.apply_model(model, label_field="ui_keypoints")

## 7. Classification

Classify the type of UI platform.

In [ ]:
model.operation = "classify"
model.prompt = "Classify the type of platform. Choose from one of: desktop, mobile, web"
dataset.apply_model(model, label_field="ui_classifications")


## 8. Using Dataset Fields as Prompts

You can use existing fields in your dataset as prompts.
In this example, we assume there's a "purpose" field that contains instructions.

In [ ]:
# If your dataset has a field called "purpose" with instructions
model.operation = "agentic"
dataset.apply_model(model, prompt_field="purpose", label_field="agentic_output")

## 9. View Results

Examine the results for the first sample.

In [ ]:
sample = dataset.first()
print(f"VQA Result: {sample.vqa_results}")
print(f"Detections: {sample.ui_detections}")
# You can view all results in the FiftyOne App with: fo.launch_app(dataset)

In [ ]:
# Visualize all results in the FiftyOne App
fo.launch_app(dataset)